# Exercise 3: Amdahl's Law and Gustafson's Law Visualizations

This notebook visualizes the scaling laws for parallel computing based on the profiling results.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import csv

# Set style for better looking plots
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11

## Load Data

In [ ]:
# Load profiling results
profiling_results = []
with open('profiling_results.csv', 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        profiling_results.append({
            'N': int(row['N']),
            'fs': float(row['fs']),
            'fp': float(row['fp'])
        })

# Load scaling results
scaling_results = []
with open('scaling_results.csv', 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        scaling_results.append({
            'N': int(row['N']),
            'Processors': int(row['Processors']),
            'fs': float(row['fs']),
            'Amdahl_Speedup': float(row['Amdahl_Speedup']),
            'Gustafson_Speedup': float(row['Gustafson_Speedup']),
            'Efficiency': float(row['Efficiency'])
        })

print(f"Loaded {len(profiling_results)} profiling results")
print(f"Loaded {len(scaling_results)} scaling results")
print(f"\nSequential fractions for different N:")
for result in profiling_results:
    print(f"  N = {result['N']:>11,}: fs = {result['fs']:.4f} ({result['fs']*100:.2f}%)")

## Question 3: Strong Scaling (Amdahl's Law)

Plot speedup curves for Amdahl's Law with measured sequential fraction.

In [ ]:
# Extract data for N=10M (representative)
n_value = 10_000_000
data_10m = [r for r in scaling_results if r['N'] == n_value]

processors = [r['Processors'] for r in data_10m]
amdahl_speedup = [r['Amdahl_Speedup'] for r in data_10m]
fs = data_10m[0]['fs']
max_speedup = 1 / fs

# Create the plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Speedup curve
ax1.plot(processors, amdahl_speedup, 'o-', linewidth=2, markersize=8, label=f'Amdahl Speedup (fs={fs:.3f})')
ax1.plot(processors, processors, '--', linewidth=2, alpha=0.5, label='Linear Speedup (Ideal)')
ax1.axhline(y=max_speedup, color='r', linestyle='--', linewidth=2, label=f'Maximum Speedup = {max_speedup:.2f}x')
ax1.set_xlabel('Number of Processors (p)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Speedup S(p)', fontsize=12, fontweight='bold')
ax1.set_title('Question 3: Amdahl\'s Law - Strong Scaling\n(Fixed Problem Size)', fontsize=14, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)
ax1.set_xscale('log', base=2)
ax1.set_xticks(processors)
ax1.set_xticklabels(processors)

# Plot 2: Efficiency
efficiency = [r['Efficiency'] for r in data_10m]
ax2.plot(processors, efficiency, 's-', linewidth=2, markersize=8, color='orange', label=f'Efficiency (fs={fs:.3f})')
ax2.axhline(y=1.0, color='g', linestyle='--', linewidth=2, alpha=0.5, label='Perfect Efficiency')
ax2.set_xlabel('Number of Processors (p)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Efficiency E(p) = S(p)/p', fontsize=12, fontweight='bold')
ax2.set_title('Parallel Efficiency vs Processors', fontsize=14, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)
ax2.set_xscale('log', base=2)
ax2.set_xticks(processors)
ax2.set_xticklabels(processors)
ax2.set_ylim([0, 1.1])

plt.tight_layout()
plt.savefig('q3_amdahl_strong_scaling.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\nKey Observations:")
print(f"  - Sequential fraction: {fs*100:.2f}%")
print(f"  - Maximum theoretical speedup: {max_speedup:.2f}x")
print(f"  - Speedup at p=64: {amdahl_speedup[-1]:.2f}x (efficiency: {efficiency[-1]*100:.1f}%)")
print(f"  - Speedup saturates around p=16-32 due to sequential bottleneck")

## Question 4: Effect of Problem Size

Compare Amdahl's Law speedup for different values of N.

In [ ]:
# Create plots for different N values
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

colors = ['blue', 'green', 'red']
markers = ['o', 's', '^']

# Plot speedup curves for each N
for i, n_val in enumerate([5_000_000, 10_000_000, 100_000_000]):
    data_n = [r for r in scaling_results if r['N'] == n_val]
    procs = [r['Processors'] for r in data_n]
    speedup = [r['Amdahl_Speedup'] for r in data_n]
    fs_val = data_n[0]['fs']
    
    label = f'N = {n_val:,} (fs={fs_val:.3f})'
    ax1.plot(procs, speedup, marker=markers[i], linewidth=2, markersize=8, 
             color=colors[i], label=label)

# Add ideal speedup line
ax1.plot(processors, processors, '--', linewidth=2, color='gray', alpha=0.5, label='Linear (Ideal)')

ax1.set_xlabel('Number of Processors (p)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Speedup S(p)', fontsize=12, fontweight='bold')
ax1.set_title('Question 4: Effect of Problem Size on Amdahl\'s Law', fontsize=14, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)
ax1.set_xscale('log', base=2)
ax1.set_xticks(processors)
ax1.set_xticklabels(processors)

# Plot sequential fraction vs N
n_values = [r['N'] for r in profiling_results]
fs_values = [r['fs'] for r in profiling_results]

ax2.plot(n_values, fs_values, 'o-', linewidth=2, markersize=10, color='purple')
ax2.axhline(y=np.mean(fs_values), color='red', linestyle='--', linewidth=2, 
            label=f'Average fs = {np.mean(fs_values):.4f}')
ax2.set_xlabel('Problem Size (N)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Sequential Fraction (fs)', fontsize=12, fontweight='bold')
ax2.set_title('Sequential Fraction vs Problem Size', fontsize=14, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)
ax2.set_xscale('log')

plt.tight_layout()
plt.savefig('q4_problem_size_effect.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\nKey Observations:")
print(f"  - Sequential fraction remains approximately constant (~30.7%) across different N")
print(f"  - All problem sizes show similar speedup curves")
print(f"  - This indicates that both sequential and parallel parts scale linearly with N")
print(f"  - The ratio of sequential to parallel work stays the same")

## Question 5: Weak Scaling (Gustafson's Law)

Compare Amdahl's Law (strong scaling) with Gustafson's Law (weak scaling).

In [ ]:
# Use N=10M data for comparison
data_10m = [r for r in scaling_results if r['N'] == 10_000_000]

processors = [r['Processors'] for r in data_10m]
amdahl_speedup = [r['Amdahl_Speedup'] for r in data_10m]
gustafson_speedup = [r['Gustafson_Speedup'] for r in data_10m]
fs = data_10m[0]['fs']

# Create comparison plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Speedup comparison
ax1.plot(processors, amdahl_speedup, 'o-', linewidth=2.5, markersize=10, 
         label=f'Amdahl\'s Law (fs={fs:.3f})', color='blue')
ax1.plot(processors, gustafson_speedup, 's-', linewidth=2.5, markersize=10, 
         label=f'Gustafson\'s Law (fs={fs:.3f})', color='green')
ax1.plot(processors, processors, '--', linewidth=2, alpha=0.5, 
         label='Linear Speedup (Ideal)', color='gray')
ax1.axhline(y=1/fs, color='red', linestyle='--', linewidth=2, alpha=0.7,
            label=f'Amdahl Max = {1/fs:.2f}x')

ax1.set_xlabel('Number of Processors (p)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Speedup S(p)', fontsize=12, fontweight='bold')
ax1.set_title('Question 5: Amdahl vs Gustafson - Scaling Laws Comparison', 
              fontsize=14, fontweight='bold')
ax1.legend(fontsize=10, loc='upper left')
ax1.grid(True, alpha=0.3)
ax1.set_xscale('log', base=2)
ax1.set_xticks(processors)
ax1.set_xticklabels(processors)

# Plot 2: Speedup ratio (Gustafson / Amdahl)
speedup_ratio = [g/a for g, a in zip(gustafson_speedup, amdahl_speedup)]
ax2.plot(processors, speedup_ratio, '^-', linewidth=2.5, markersize=10, 
         color='purple', label='Gustafson / Amdahl Ratio')
ax2.axhline(y=1.0, color='gray', linestyle='--', linewidth=2, alpha=0.5)

ax2.set_xlabel('Number of Processors (p)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Speedup Ratio (Gustafson / Amdahl)', fontsize=12, fontweight='bold')
ax2.set_title('Advantage of Weak Scaling over Strong Scaling', 
              fontsize=14, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)
ax2.set_xscale('log', base=2)
ax2.set_xticks(processors)
ax2.set_xticklabels(processors)

plt.tight_layout()
plt.savefig('q5_amdahl_vs_gustafson.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\nKey Observations:")
print(f"\nAmdahl's Law (Strong Scaling - Fixed Problem Size):")
print(f"  - Speedup saturates at ~{max(amdahl_speedup):.2f}x")
print(f"  - Limited by sequential fraction (fs = {fs*100:.2f}%)")
print(f"  - Adding more processors beyond p=32 provides minimal benefit")
print(f"\nGustafson's Law (Weak Scaling - Growing Problem Size):")
print(f"  - Speedup grows nearly linearly: {gustafson_speedup[-1]:.2f}x at p=64")
print(f"  - Assumes problem size scales with number of processors")
print(f"  - More optimistic view of parallel computing potential")
print(f"\nComparison:")
print(f"  - At p=64: Gustafson is {speedup_ratio[-1]:.2f}x better than Amdahl")
print(f"  - Amdahl: Good for fixed workloads (analyzing specific dataset)")
print(f"  - Gustafson: Good for scalable workloads (higher resolution simulations)")

## Summary Table

In [ ]:
# Create summary table
print("="*100)
print("COMPREHENSIVE SCALING ANALYSIS SUMMARY")
print("="*100)
print(f"\nProgram: Parallel Computing Workload with Sequential Bottleneck")
print(f"Sequential Part: add_noise() function (data-dependent loop)")
print(f"Parallelizable Parts: init_b(), compute_addition(), reduction()")
print(f"\nMeasured Sequential Fraction: fs ≈ {fs*100:.2f}%")
print(f"Measured Parallelizable Fraction: fp ≈ {(1-fs)*100:.2f}%")

print(f"\n{'='*100}")
print(f"{'Processors':<12} {'Amdahl S(p)':<15} {'Gustafson S(p)':<15} {'Efficiency':<15} {'G/A Ratio':<12}")
print(f"{'='*100}")

for i, p in enumerate(processors):
    print(f"{p:<12} {amdahl_speedup[i]:<15.3f} {gustafson_speedup[i]:<15.3f} "
          f"{amdahl_speedup[i]/p:<15.3f} {gustafson_speedup[i]/amdahl_speedup[i]:<12.2f}")

print(f"{'='*100}")
print(f"\nMaximum Theoretical Speedup (Amdahl): {1/fs:.2f}x")
print(f"\nConclusion:")
print(f"  - The program is significantly limited by its sequential portion (~30.7%)")
print(f"  - Strong scaling (Amdahl) shows diminishing returns beyond 8-16 processors")
print(f"  - Weak scaling (Gustafson) shows better scalability when problem size grows")
print(f"  - To improve performance: reduce or eliminate the sequential add_noise() dependency")

## Additional Analysis: Speedup per Function

Break down which functions could benefit most from parallelization.

In [ ]:
# Calculate contribution of each function
prof = profiling_results[1]  # Use N=10M data

# From callgrind data (N=10M)
functions = {
    'add_noise': {'instructions': 50_000_002, 'parallelizable': False},
    'init_b': {'instructions': 27_500_009, 'parallelizable': True},
    'compute_addition': {'instructions': 60_000_004, 'parallelizable': True},
    'reduction': {'instructions': 25_000_006, 'parallelizable': True}
}

total = sum(f['instructions'] for f in functions.values())

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Pie chart of instruction distribution
labels = list(functions.keys())
sizes = [f['instructions'] for f in functions.values()]
colors_pie = ['#ff6b6b' if not f['parallelizable'] else '#51cf66' for f in functions.values()]
explode = (0.1, 0, 0, 0)  # Explode add_noise to highlight

ax1.pie(sizes, explode=explode, labels=labels, colors=colors_pie, autopct='%1.1f%%',
        shadow=True, startangle=90, textprops={'fontsize': 11, 'fontweight': 'bold'})
ax1.set_title('Instruction Distribution by Function\n(Red = Sequential, Green = Parallelizable)', 
              fontsize=14, fontweight='bold')

# Bar chart of potential speedup per function
func_names = list(functions.keys())
percentages = [f['instructions']/total*100 for f in functions.values()]
bar_colors = ['red' if not f['parallelizable'] else 'green' for f in functions.values()]

bars = ax2.bar(func_names, percentages, color=bar_colors, alpha=0.7, edgecolor='black', linewidth=2)
ax2.set_xlabel('Function', fontsize=12, fontweight='bold')
ax2.set_ylabel('Percentage of Total Instructions (%)', fontsize=12, fontweight='bold')
ax2.set_title('Function Contribution to Runtime', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.1f}%', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.savefig('function_breakdown.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nFunction Analysis:")
print("="*60)
for name, data in functions.items():
    pct = data['instructions']/total*100
    status = "SEQUENTIAL" if not data['parallelizable'] else "PARALLELIZABLE"
    print(f"{name:<20} {pct:>6.2f}%  [{status}]")
print("="*60)